In [9]:
!pip show streamlit
!pip install nbconvert


Name: streamlit
Version: 1.38.0
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: C:\Users\isa-m\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
   ---------------------------------------- 0.0/257.4 kB ? eta -:--:--
   --------- ------------------------------ 61.4/257.4 kB 1.7 MB/s eta 0:00:01
   ----------------------- ---------------- 153.6/257.4 kB 1.5 MB/s eta 0:00:01
   ------------------------------------ --- 235.5/257.4 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 257.4/257.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/162.8 kB ? eta


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\isa-m\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!pip install streamlit



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\isa-m\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

# Cargar el archivo CSV
file_path = './CVD_cleaned.csv'
df = pd.read_csv(file_path)

# Mostrar las primeras filas del DataFrame
df.head()



,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


In [6]:
import sqlite3

# Conectar a la base de datos SQLite (se creará si no existe)
conn = sqlite3.connect('CVD_data.db')

# Guardar el DataFrame en la base de datos
df.to_sql('health_data', conn, if_exists='replace', index=False)

# Confirmar que se ha guardado
print("DataFrame guardado en la base de datos SQLite.")

# Cerrar la conexión
conn.close()


DataFrame guardado en la base de datos SQLite.


In [7]:
import streamlit as st
import pandas as pd
import sqlite3

# Función para conectarse a la base de datos SQLite
def connect_db():
    conn = sqlite3.connect('CVD_data.db')
    return conn

# Función para leer los datos de la base de datos
def leer_datos():
    conn = connect_db()
    df = pd.read_sql_query("SELECT * FROM health_data", conn)
    conn.close()
    return df

# Función para insertar nuevos registros
def insertar_dato(general_health, checkup, exercise, heart_disease, skin_cancer):
    conn = connect_db()
    query = """
    INSERT INTO health_data (General_Health, Checkup, Exercise, Heart_Disease, Skin_Cancer)
    VALUES (?, ?, ?, ?, ?)
    """
    conn.execute(query, (general_health, checkup, exercise, heart_disease, skin_cancer))
    conn.commit()
    conn.close()

# Función para actualizar un registro
def actualizar_dato(old_health, new_health):
    conn = connect_db()
    query = "UPDATE health_data SET General_Health = ? WHERE General_Health = ?"
    conn.execute(query, (new_health, old_health))
    conn.commit()
    conn.close()

# Función para eliminar un registro
def eliminar_dato(health_condition):
    conn = connect_db()
    query = "DELETE FROM health_data WHERE General_Health = ?"
    conn.execute(query, (health_condition,))
    conn.commit()
    conn.close()

# Título de la aplicación
st.title("Visualización de Datos de Salud - CVD")

# Mostrar los datos
st.subheader("Datos actuales")
df = leer_datos()
st.dataframe(df)

# Formulario para agregar nuevos datos
st.subheader("Agregar nuevo registro")
with st.form("Agregar nuevo"):
    general_health = st.selectbox("General Health", df['General_Health'].unique())
    checkup = st.text_input("Checkup")
    exercise = st.text_input("Exercise")
    heart_disease = st.text_input("Heart Disease")
    skin_cancer = st.text_input("Skin Cancer")
    submitted = st.form_submit_button("Agregar")
    
    if submitted:
        insertar_dato(general_health, checkup, exercise, heart_disease, skin_cancer)
        st.success("Registro agregado exitosamente")

# Formulario para actualizar datos
st.subheader("Actualizar registro")
with st.form("Actualizar"):
    old_health = st.selectbox("Seleccionar General Health", df['General_Health'].unique())
    new_health = st.text_input("Nuevo estado de salud")
    update_submitted = st.form_submit_button("Actualizar")
    
    if update_submitted:
        actualizar_dato(old_health, new_health)
        st.success("Registro actualizado exitosamente")

# Formulario para eliminar datos
st.subheader("Eliminar registro")
with st.form("Eliminar"):
    health_condition = st.selectbox("Seleccionar General Health a eliminar", df['General_Health'].unique())
    delete_submitted = st.form_submit_button("Eliminar")
    
    if delete_submitted:
        eliminar_dato(health_condition)
        st.success("Registro eliminado exitosamente")


2024-10-01 16:27:26.473 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 16:27:26.653 
  command:

    streamlit run C:\Users\isa-m\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-10-01 16:27:26.653 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 16:27:26.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 16:27:26.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 16:27:28.309 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 16:27:28.322 Thread 'MainThread': missing ScriptRunContext! This w

In [10]:
!jupyter nbconvert --to script SQL_Martinez,Monsalve,Erazo.ipynb



[NbConvertApp] Converting notebook SQL_Martinez,Monsalve,Erazo.ipynb to script
[NbConvertApp] Writing 3754 bytes to SQL_Martinez,Monsalve,Erazo.py


In [11]:
!streamlit run SQL_Martinez,Monsalve,Erazo.py


^C
